## Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import joblib
from sklearn.metrics import f1_score
import math

#### defining function 1

In [2]:
def function1(path):
    
    """
    this function takes path of raw test data as input and
    prints class label for the point
    """
    
    test_df = pd.read_csv(path)


    test_df = test_df.drop(["bn_000","bo_000","bp_000","bq_000","br_000"],axis=1)# drop these columns

    test_df[["ab_000","cr_000"]] = test_df[["ab_000","cr_000"]].fillna(1) # fill nan values with 1

    test_df.loc[test_df["ab_000"] != 1, "ab_000"] = 0# fill everything except 1 with zero
    test_df.loc[test_df["cr_000"] != 1, "cr_000"] = 0

    test_df_70 = test_df[["ab_000","cr_000"]]
    
    knn_imputer = "knn_imputer.pkl"
    new_imputer = joblib.load(knn_imputer)

    median = pd.read_csv("median.csv")
    miss_column_10 = np.load("median_column.npy",allow_pickle=True)
    miss_column_10_70 = np.load("knn_column.npy",allow_pickle=True)
    
    # replace values in those columns which have missing value less than 10% by median value of train data for corresponding feature
    test_df[miss_column_10] = test_df[miss_column_10].fillna(median["0"].median())
    test_df_10 = test_df[miss_column_10]
    
    test_df_10_70 = pd.DataFrame(new_imputer.transform(test_df[miss_column_10_70]))
    test_df_10_70.columns = miss_column_10_70
    test_df_10_70.isnull().values.any()
    
    test_cleaned = pd.concat([test_df_10,test_df_10_70,test_df_70],axis=1)
    
    top_10 = ['bi_000', 'ay_002', 'ay_006', 'cc_000', 'ay_008', 'al_000', 'ag_001','ag_002', 'bj_000', 'ay_005'] 
    top_4 = ["ay_006", "cc_000", "ay_008", "bj_000"]
    top_4_median = [165116.0, 2112040.0, 92906.0, 154640.0]
    
    for i in top_10:
        
        temp1 = i + "_sin"
        temp2 = i + "_log"
    
        test_cleaned[temp2] = test_cleaned[i].apply(lambda x: math.log(x+1))
        test_cleaned[temp1] = test_cleaned[i].apply(lambda x: math.sin(x))
        
    for i in range(4):
    
        temp1 = top_4[i] + "_median"
        test_cleaned[temp1] = test_cleaned[top_4[i]] - top_4_median[i]
        
    my_model = "model.pkl"
    clf = joblib.load(my_model)
    y_test_pred = clf.predict(test_cleaned)
    print("prediction is is - ",y_test_pred[0])

#### testing function 1

In [3]:
function1("single_data.csv")

prediction is is -  0


#### defining function 2

In [7]:
def function2(path):
    
    """
    this function takes path of raw test data as input and
    print(macro f1 score)
    """
    
    test_df = pd.read_csv(path)#nrows=1000)
    # droping useless columns
    test_df = test_df.drop(["bn_000","bo_000","bp_000","bq_000","br_000"],axis=1)# drop these columns
    test_df[["ab_000","cr_000"]] = test_df[["ab_000","cr_000"]].fillna(1) # fill nan values with 1

    test_df.loc[test_df["ab_000"] != 1, "ab_000"] = 0# fill everything except 1 with zero
    test_df.loc[test_df["cr_000"] != 1, "cr_000"] = 0

    test_df_70 = test_df[["ab_000","cr_000"]]
    
    # load knn imputer
    knn_imputer = "knn_imputer.pkl"
    new_imputer = joblib.load(knn_imputer)

    median = pd.read_csv("median.csv")
    miss_column_10 = np.load("median_column.npy",allow_pickle=True)
    miss_column_10_70 = np.load("knn_column.npy",allow_pickle=True)
    
    # replace values in those columns which have missing value less than 10% by median value of train data for corresponding feature
    test_df[miss_column_10] = test_df[miss_column_10].fillna(median["0"].median())

    test_df_10 = test_df[miss_column_10]
    
    # replace values in those columns which have missing value between 10% and 70% by knn imputer
    test_df_10_70 = pd.DataFrame(new_imputer.transform(test_df[miss_column_10_70]))
    test_df_10_70.columns = miss_column_10_70
    
    test_cleaned = pd.concat([test_df_10,test_df_10_70,test_df_70],axis=1)
    test_cleaned["class"] = test_df["class"]
    
    top_10 = ['bi_000', 'ay_002', 'ay_006', 'cc_000', 'ay_008', 'al_000', 'ag_001','ag_002', 'bj_000', 'ay_005'] 
    top_4 = ["ay_006", "cc_000", "ay_008", "bj_000"]
    top_4_median = [165116.0, 2112040.0, 92906.0, 154640.0]
    
    # feature engg
    for i in top_10:
        
        temp1 = i + "_sin"
        temp2 = i + "_log"
    
        test_cleaned[temp2] = test_cleaned[i].apply(lambda x: math.log(x+1))
        test_cleaned[temp1] = test_cleaned[i].apply(lambda x: math.sin(x))
        
    for i in range(4):
    
        temp1 = top_4[i] + "_median"
        test_cleaned[temp1] = test_cleaned[top_4[i]] - top_4_median[i]
        
    # loading best model
    my_model = "model.pkl"
    clf = joblib.load(my_model)
    
    y_test = test_cleaned["class"]
    x_test = test_cleaned.drop(["class"],axis=1)
    y_test_pred = clf.predict(x_test)
    
    f1_test = f1_score(y_test,y_test_pred, average='macro')
    print("Test macro f1 is - ",f1_test)

#### testing function 2

In [8]:
function2("test.csv")

Test macro f1 is -  0.9144926251071502


## working demo link

https://youtu.be/ODcfNROq8ck